In [1]:
import torch
import numpy as np
import nir

In [2]:
m = torch.load("model_ref_20230827_182756.pt", map_location="cpu")
m

OrderedDict([('fc1.weight',
              tensor([[-3.5701e-01, -5.6863e-01, -1.1815e+00,  ..., -8.3889e-01,
                       -1.9059e-01, -3.2760e-01],
                      [-7.0992e-01, -8.6060e-01, -6.3683e-01,  ..., -1.2058e+00,
                       -2.3467e-01, -5.9531e-01],
                      [-3.0828e+00, -2.8350e+00, -3.4598e+00,  ..., -2.9395e+00,
                       -2.3283e+00, -2.1144e+00],
                      ...,
                      [-6.2610e-01, -1.1959e+00, -8.1784e-01,  ..., -7.1882e-01,
                       -6.2435e-01, -1.0815e+00],
                      [ 3.3846e-02,  3.9589e-03, -2.2722e-01,  ..., -1.5527e-01,
                       -3.9545e+00, -5.4881e+00],
                      [-1.4432e+00, -1.7614e+00, -1.9475e+00,  ..., -1.4423e+00,
                       -1.9547e+00, -1.1560e+00]])),
             ('fc1.bias',
              tensor([-9.8048e-01, -1.0654e+00, -3.8092e+00, -5.5239e-01, -3.1662e-01,
                      -2.1900e+00, -7.3272e

In [3]:
[x for x in m.keys()]

['fc1.weight',
 'fc1.bias',
 'lif1.threshold',
 'lif1.graded_spikes_factor',
 'lif1.reset_mechanism_val',
 'lif1.beta',
 'lif1.recurrent.weight',
 'lif1.recurrent.bias',
 'fc2.weight',
 'fc2.bias',
 'lif2.threshold',
 'lif2.graded_spikes_factor',
 'lif2.reset_mechanism_val',
 'lif2.beta']

In [4]:
m["lif1.graded_spikes_factor"]

tensor(1.)

In [5]:
gen = lambda *x: torch.randn(*x).numpy()
n1 = 16
n2 = 63
g = nir.NIRGraph(
    nodes = {
        "fc1": nir.Affine(weight=gen(n1, 24), bias=gen(n1)),
        "lif1": nir.CubaLIF(w_in=torch.eye(n1).numpy(),
                            tau_mem=torch.full((n1,), 1).numpy(),
                            tau_syn=torch.full((n1,), 1).numpy(),
                            r=torch.full((n1,), 1).numpy(),
                            v_leak=torch.zeros((n1,)).numpy(),
                            v_threshold=torch.full((n1,), 1).numpy()),
        "lif1r": nir.Affine(weight=gen(n2, n1), bias=gen(n2)),
        "fc2": nir.Affine(weight=gen(n2, n2), bias=gen(n2)),
        "lif2": nir.CubaLIF(w_in=torch.eye(n2).numpy(),
                            tau_mem=torch.full((n2,), 1).numpy(),
                            tau_syn=torch.full((n2,), 1).numpy(),
                            r=torch.full((n2,), 1).numpy(),
                            v_leak=torch.zeros((n2,)).numpy(),
                            v_threshold=torch.ones((n2,)).numpy()),
    },
    edges = [("fc1", "lif1"), ("lif1", "lif1r"), ("lif1r", "lif1"), ("lif1", "fc2"), ("fc2", "lif2")]
)

nir.write("braille.nir", g)

In [6]:
nir.read("braille.nir")

NIRGraph(nodes={'fc1': Affine(weight=array([[ 0.19632688, -0.15066636,  0.79383886, -0.6918571 , -1.1997273 ,
         0.03167971,  0.09418193,  1.2134423 ,  1.015343  , -0.00683297,
        -0.24639373,  0.23417024,  1.6612574 ,  1.4080112 , -0.72425294,
         0.83691615,  0.52794087,  2.000234  , -0.18228869,  0.911659  ,
         1.1834587 , -0.07246201,  0.29142314, -0.12610467],
       [ 0.1005088 ,  0.25827315, -0.71275324,  1.2558497 , -0.08012172,
        -0.17658281,  0.7774217 , -0.59269005,  0.84339434,  0.4337682 ,
         0.05873786,  0.23892304,  0.06781289, -1.0675094 , -0.5197273 ,
        -0.8751303 ,  0.7234639 , -0.6403121 , -0.7771464 ,  0.3853149 ,
         1.2668241 , -1.234899  ,  0.55707663,  1.9268793 ],
       [ 1.1873337 , -0.78655595, -0.5718737 ,  0.03339281,  0.16578901,
         1.0672818 ,  0.5902614 , -0.68486565, -0.7119151 ,  1.0009782 ,
        -0.821502  ,  0.12713674, -1.7987015 , -1.304858  , -0.6131657 ,
         0.2677764 , -1.0950854 , -0.3

In [7]:
import norse

In [8]:
norse.torch.from_nir(g)

GraphExecutor(
  (fc1): Linear(in_features=24, out_features=16, bias=True)
  (lif1): LIFCell(p=LIFParameters(tau_syn_inv=tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), tau_mem_inv=tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), v_leak=tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), v_th=tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), v_reset=tensor(0.), method='super', alpha=tensor(100.)), dt=0.001)
  (lif1r): Linear(in_features=16, out_features=63, bias=True)
  (fc2): Linear(in_features=63, out_features=63, bias=True)
  (lif2): LIFCell(
    p=LIFParameters(tau_syn_inv=tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
            1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
            1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
            1., 1., 1., 1., 1., 1., 1., 1., 1.]), tau_mem_inv=tensor([1., 1., 1., 1., 1